In [1]:
import re
import json
import logging
import asyncio
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment
from market_agents.memory.agent_storage.agent_storage_api_utils import AgentStorageAPIUtils
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.economics.econ_agent import EconomicAgent
from market_agents.agents.personas.persona import load_persona_from_file


In [2]:
config_path = Path("../market_agents/memory/storage_config.yaml")
config = load_config_from_yaml(str(config_path))

# Initialize embedding service and storage API utils
embedder = MemoryEmbedder(config)
storage_utils = AgentStorageAPIUtils(
    config=config,
    logger=logging.getLogger("storage_api")
)

hamlet_kb = MarketKnowledgeBase(
    config=config,
    table_prefix="hamlet_kb"
)
await hamlet_kb.initialize()

kb_agent = KnowledgeBaseAgent(market_kb=hamlet_kb)


INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Initializing Agent Storage API Utils with URL: http://localhost:8001
INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Sending create tables request with payload: {'table_type': 'knowledge', 'agent_id': None, 'table_prefix': 'hamlet_kb'}


In [18]:
def document_reader(file_path: str) -> str:
    """
    Read content from PDF or TXT files and return cleaned text
    
    Args:
        file_path (str): Path to the document file
        
    Returns:
        str: Cleaned text content from the document
    """
    text_content = ""
    
    if file_path.endswith('.pdf'):
        pdf_reader = PdfReader(file_path)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            cleaned_text = re.sub(r'FTLN \d+', '', page_text)
            cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
            text_content += cleaned_text
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or TXT file.")
            
    return text_content

# Read document content
file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/hamlet_PDF_FolgerShakespeare.pdf"
text_content = document_reader(file_path)

In [ ]:
# Store in knowledge base with metadata
knowledge_id = await hamlet_kb.ingest_knowledge(
    text=text_content,
    metadata={
        "source": "Hamlet",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")


In [ ]:
# Test some queries
async def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = await kb_agent.retrieve(
        query=question
    )
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    await query_hamlet(query)

In [3]:
teacher_persona = load_persona_from_file('../market_agents/agents/personas/generated_personas/Martin_Robinson.yaml')
chat_env = ChatEnvironment(name="hamlet_chat")

econ_agent = EconomicAgent(
    generate_wallet=True,
    initial_holdings={
        "ETH": 1.0,
        "USDC": 1000.0
    }
)

hamlet_agent = await MarketAgent.create(
    storage_utils=storage_utils,
    agent_id="hamlet_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
    econ_agent=econ_agent,
    persona=teacher_persona
)

# Add a test message to the chat
hamlet_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
chat_env.mechanism.add_user_message(hamlet_agent.task)

# Generate a response
perception = await hamlet_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await hamlet_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await hamlet_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))


INFO:market_agents.economics.econ_models:[AgentWallet] Auto-generating Ethereum wallet.
INFO:market_agents.economics.econ_models:[AgentWallet] Generated Ethereum wallet: 0xcCAd21451e19c81bbf9fCea9b1948de2955f9A85



Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "hamlet_agent", "action": {"thoughts": [{"reasoning": "The user has asked about Hamlet\'s \'To be or not to be\' soliloquy, a significant part of the play that explores existential themes. My previous responses have highlighted these themes effectively, and I should continue this approach."}, {"reasoning": "The retrieved documents provide context about existential themes, madness, and human nature, which are relevant to the soliloquy. I should integrate these broader themes into my response to provide a comprehensive answer."}, {"reasoning": "Including direct quotes from the soliloquy will add depth and clarity to my explanation. This will help the user understand the philosophical nature of the soliloquy."}], "final_answer": "Hamlet\'s famous \'To be or not to be\' soliloquy is a profound philosophical reflection on existence, pondering the nature of life and death. In this soliloquy, Hamlet con

In [4]:
# Print all accessible instance attributes of the hamlet_agent
for attr_name, attr_value in hamlet_agent.__dict__.items():
    value_str = str(attr_value).replace('\n', ' ')
    print(f"{attr_name}: {value_str}")

id: hamlet_agent
role: Buyer
persona: You are Martin Robinson. Martin Robinson is a Non-binary who participates in the market. They are 56 years old and Their occupation is Teacher. Their investment experience are Novice, and they have a Aggressive risk appetite.  ## Demographic Characteristics - Age: 56 - Gender: Non-binary - Education Level: Master's - Occupation: Teacher - Income Bracket: Low - Geographic Location: Urban  ## Economic Attributes - Spending Habits: Frugal - Saving Preferences: Medium - Risk Tolerance: 0.8 - Investment Experience: Novice  ## Personality Traits - Communication Style: Friendly - Decision Making Style: Emotional - Openness: 0.29 - Conscientiousness: 0.87 - Extraversion: 0.3 - Agreeableness: 0.88 - Neuroticism: 0.25  ## Hobbies and Interests Art, Sports, Gardening  ## Routines Grade assignments, Teach classes, Meet with parents  ## Skills Instruction, Communication, Classroom management  ## Dynamic Attributes - Current Mood: Neutral - Recent Life Events: S

In [13]:
from pprint import pformat
print(pformat(repr(hamlet_agent.economic_agent), indent=2, width=100))

EconomicAgent(id='hamlet_agent',
  rewards=[],
  total_reward=0.0,
  wallet=AgentWallet(chain='ethereum',
  address='0xcCAd21451e19c81bbf9fCea9b1948de2955f9A85',
  private_key='35c30618303b246b9f8c4ac6c7ff66dbe900b536675f161b228b83a181a913b6'),
  holdings=Portfolio(token_balances={'ETH': 1.0,
  'USDC': 1000.0},
  transaction_history=[]),
  generate_wallet=True)


In [14]:
hamlet_agent.prompt_context.history

[{'role': 'user',
  'content': 'Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n-   content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  timestamp: 2025-01-29 09:59:24\n  role: user\n\n## Recent Cognitive Memories\n-   cognitive_step: action\n  content: {"agent_id": "hamlet_agent", "action": {"thoughts": [{"reasoning": "The user has asked about Hamlet\'s \'To be or not to be\' soliloquy, a significant part of the play that explores existential themes. My previous responses have highlighted these themes effectively, and I should continue this approach."}, {"reasoning": "The retrieved documents provide context about existential themes, madness, and human nature, which are relevant to the soliloquy. I should integrate these broader themes into my response to provide a comprehensive answer."}, {"reasoning": "Including direct quotes from the soliloquy will add depth and clarity to my explana

In [15]:
hamlet_agent.interactions

[{'id': 'hamlet_agent',
  'name': 'Buyer',
  'system': {'role': 'system',
   'content': "Role: You are a Buyer agent. You are a long-horizon task execution agent based on verbal reinforcement learning. The current date and time is 2025-01-29 09:59:30.\n\nPersona: You have the following personal background:\nYou are Martin Robinson. Martin Robinson is a Non-binary who participates in the market. They are 56 years old and Their occupation is Teacher. Their investment experience are Novice, and they have a Aggressive risk appetite.\n\n## Demographic Characteristics\n- Age: 56\n- Gender: Non-binary\n- Education Level: Master's\n- Occupation: Teacher\n- Income Bracket: Low\n- Geographic Location: Urban\n\n## Economic Attributes\n- Spending Habits: Frugal\n- Saving Preferences: Medium\n- Risk Tolerance: 0.8\n- Investment Experience: Novice\n\n## Personality Traits\n- Communication Style: Friendly\n- Decision Making Style: Emotional\n- Openness: 0.29\n- Conscientiousness: 0.87\n- Extraversion